# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import itertools

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [7]:
df = pd.read_csv("../data/day-of-week-not-scaled.csv")
df['dayofweek'] = pd.read_csv("../data/dayofweek.csv")['dayofweek']
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [8]:
X = df.drop(columns=['dayofweek'])
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21, stratify=y, test_size=0.2)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [4]:
svc = SVC()
param_grid = {
    'kernel' : ['linear', 'rbf', 'sigmoid'],
    'C' : [0.01, 0.1, 1, 1,5, 5, 10],
    'gamma' : ['scale', 'auto'],
    'class_weight' : ['balanced', None],
    'random_state' : [21],
    'probability' : [True]
}
gs = GridSearchCV(estimator=svc,param_grid=param_grid, verbose=3, n_jobs=-1).fit(X_train, y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[CV 2/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=linear, probability=True, random_state=21;, score=0.344 total time=   0.4s
[CV 4/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=linear, probability=True, random_state=21;, score=0.346 total time=   0.4s
[CV 3/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=linear, probability=True, random_state=21;, score=0.337 total time=   0.4s
[CV 5/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=linear, probability=True, random_state=21;, score=0.346 total time=   0.4s
[CV 1/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=linear, probability=True, random_state=21;, score=0.326 total time=   0.4s
[CV 1/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=rbf, probability=True, random_state=21;, score=0.233 total time=   0.5s
[CV 2/5] END C=0.01, class_weight=balanced, gamma=scale, kernel=rbf, probability=True, random_state=21;, score=0.233 total time=   0.5s
[CV 4/5] END C=0.01, class_weight

In [22]:
gs.best_params_

{'C': 10,
 'class_weight': None,
 'gamma': 'auto',
 'kernel': 'rbf',
 'probability': True,
 'random_state': 21}

In [38]:
params = pd.DataFrame(gs.cv_results_).filter(regex='^param_|rank|mean_test_score') 
params.sort_values(by='rank_test_score').head()


,param_C,param_class_weight,param_gamma,param_kernel,param_probability,param_random_state,mean_test_score,rank_test_score
82,10.0,None,auto,rbf,True,21,0.876109,1
76,10.0,balanced,auto,rbf,True,21,0.863500,2
58,5.0,None,auto,rbf,True,21,0.816018,3
70,5.0,None,auto,rbf,True,21,0.816018,3
64,5.0,balanced,auto,rbf,True,21,0.808608,5


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [48]:
tree = DecisionTreeClassifier()

param_grid = {
    'max_depth' : range(1, 49, 2),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini'],
    'random_state' : [21]
}
gs = GridSearchCV(estimator=tree,param_grid=param_grid, verbose=1, n_jobs=-1).fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [49]:
gs.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 21,
 'random_state': 21}

In [ ]:
params = pd.DataFrame(gs.cv_results_).filter(regex='^param_|rank|mean_test_score') 
params.sort_values(by='rank_test_score').head()

,param_class_weight,param_criterion,param_max_depth,param_random_state,mean_test_score,rank_test_score
34,balanced,gini,21,21,0.873121,1
40,balanced,gini,33,21,0.873116,2
47,balanced,gini,47,21,0.873116,2
41,balanced,gini,35,21,0.873116,2
43,balanced,gini,39,21,0.873116,2


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [20]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators' : [5, 10, 50, 100],
    'max_depth' : range(1, 49),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini'],
    'random_state' : [21]
}
gs = GridSearchCV(estimator=rf, param_grid=param_grid, verbose=2, n_jobs=-1).fit(X_train, y_train)

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=5, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=5, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=10, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=5, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=10, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=10, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=10, random_state=21; total time=   0.0s
[CV] END class_weight=balanced, criterion=entropy, max_depth=1, n_estimators=10, random_state=21; total time=   0.0s
[CV

In [53]:
params = pd.DataFrame(gs.cv_results_).filter(regex='^param_|rank|mean_test_score') 
params.sort_values(by='rank_test_score').head()

,param_class_weight,param_criterion,param_max_depth,param_n_estimators,param_random_state,mean_test_score,rank_test_score
686,None,gini,28,50,21,0.904290,1
699,None,gini,31,100,21,0.903547,2
310,balanced,gini,30,50,21,0.902817,3
326,balanced,gini,34,50,21,0.902809,4
727,None,gini,38,100,21,0.902806,5


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [5]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators' : [5, 10, 50, 100],
    'max_depth' : range(1, 49),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini'],
    'random_state' : [21]
}
combinations = list(itertools.product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['class_weight'],
    param_grid['criterion'],
    param_grid['random_state']
))

results = []

In [9]:
for n_estimators, max_depth, class_weight, criterion, random_state in tqdm(combinations):
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                class_weight=class_weight, criterion=criterion, random_state=random_state)
    scores = cross_val_score(rf, X_train, y_train, cv=5, n_jobs=-1)
    results.append({
        'n_estimators' : n_estimators,
        'max_depth' : max_depth,
        'class_weight' : class_weight,
        'criterion' : criterion,
        'random_state' : random_state,
        'mean_accuracy' : np.mean(scores),
        'std_accuracy' : np.std(scores)
    })

  0%|          | 0/768 [00:00<?, ?it/s]

In [10]:
pd.DataFrame(results).sort_values(by='mean_accuracy', ascending=False)

,n_estimators,max_depth,class_weight,criterion,random_state,mean_accuracy,std_accuracy
495,50,28,None,gini,21,0.904290,0.010961
699,100,31,None,gini,21,0.903547,0.014380
501,50,30,balanced,gini,21,0.902817,0.013554
517,50,34,balanced,gini,21,0.902809,0.013010
735,100,40,None,gini,21,0.902806,0.010460
...,...,...,...,...,...,...,...
2,5,1,None,entropy,21,0.353832,0.016467
4,5,2,balanced,entropy,21,0.353110,0.021165
5,5,2,balanced,gini,21,0.346419,0.029749
1,5,1,balanced,gini,21,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [11]:
best_rf = RandomForestClassifier(n_estimators=50, max_depth=28, class_weight=None, 
                                 criterion='gini', random_state=21).fit(X_train, y_train)
y_best_rf_pred = best_rf.predict(X_test)
accuracy_score(y_test, y_best_rf_pred)

0.9289940828402367